<a href="https://colab.research.google.com/github/SoumyaShreeram/Microlensing_with_NeuralNets/blob/master/02i_Preprocessing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02. Functions for preprocessing, generation, and plotting data

Preprocessing includes: Sampling, initialization, setting the filename, and loading data

Author: Soumya Shreeram <br>
Script adapted from: Millon Martin & Kevin Müller <br>
Date: 23rd February 2020 <br>


In [0]:
from google.colab import drive
import os
import pickle
import importlib
import itertools
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
import random
import os
from IPython.display import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, InputSpec
from tensorflow.python.keras.layers import Conv1D, Conv2D
from tensorflow.python.keras.layers import MaxPooling1D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Add, BatchNormalization, Concatenate
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer

### 1. Sampling, Initialization, loading data

In [0]:
def setSamplingParameters(v_t, data_dir, euler_sampling):
  """
  Function defines the values for sampling parameters and no. of pixels 

  Inputs:
  @v_t :: transverse velocity
  @euler_sampling :: data samples taken by the Euler telescope
  @data_dir :: directory with the model light-curves for inputted v_t

  Retutns:
  @n_sample :: no. of samples in the training set
  @n_sample_max :: option to reduce no. of samples in training set
  @n_pix :: no. of pixels, i.e. no. of data-points in the light-curve
  """
  if v_t == 500:
    if euler_sampling: 
      n_sample, n_sample_max, n_pix = 1000, 1000, 955
    else:
      n_sample, n_sample_max, n_pix = 5000, 5000, 955

  else:
    n_sample, n_sample_max, n_pix = 20000, 3000, 1137
  sample_params = [n_sample, n_sample_max, n_pix]
  return sample_params

def initializer(r_0, sample_params):
  """
  Function defines the class names, categories, and initializes the data arrays
  Input:
  @r_0 :: arr with all the scale radii of the background quasar
  @sample_params :: arr containing defined values for the sampled data

  Returns:
  @l_curves :: data to store the light curves
  @class_cat :: 2D array containing the classes and categories to be classified
  @out_categories :: output array with the categories of light cureves
  @out_radii ::  output array with all the radii of the light curves
  """
  # generate categories and class names 
  classes = [str(radius) for radius in r_0]
  categories = np.arange(len(r_0))

  # initialize data arrays to be classified
  l_curves = np.zeros((sample_params[0]*len(r_0), sample_params[2], 1))
  out_catergories = np.zeros(sample_params[0]*len(r_0))
  out_radii = np.zeros(sample_params[0]*len(r_0))

  class_cat = [classes, categories]
  return class_cat, l_curves, out_catergories, out_radii

def getFilename(data_dir, r, v_t, sample_params, euler_sampling):
  """
  Fuctions defines the file name based on the inputted transverse velocity
  @data_dir :: path to the directory containing the data
  @r :: arr containing the scale radii
  @v_t :: input value for the transverse velocity
  """
  if v_t == 300:
    filename = data_dir + 'v300/simLC_A-B_n%d_v300_R'%sample_params[0] + str(r)  + '_M0,3.pkl'
  if v_t == 500 and euler_sampling == True:
    filename = data_dir + 'v500_euler/simLC_A-B_n%d_v500_R'%sample_params[0] + str(r)  + '_M0,3.pkl'
  if v_t == 500 and euler_sampling == False:
    filename = data_dir + 'v500/simLC_A-B_n%d_v500_R'%sample_params[0] + str(r)  + '_M0,3.pkl'
  return filename

def loadData(filename, l_curves, sample_params, r, euler_sampling):
  """
  Function loads the data from the data files 
  Input:
  @l_curves :: empty and initialized arr to hold light-curve info
  @sample_params :: arr containing defined values for the sampled data 
  @r :: iterating variable over all the scale radii

  Returns @l_curves :: fills the data array with required no. of light curves
  """
  l_curve_file = open(filename, 'rb')
  l_curve_data = pickle.load(l_curve_file, encoding='latin1')
  l_curve_file.close()

  # storing the save data in separate arrays
  if euler_sampling:
    lc, mjhd, err_mag_ml  = l_curve_data[0], l_curve_data[1], l_curve_data[2]
  else:
    lc = l_curve_data

  # counter makes sure background is eliminated
  count = 0

  # iterating over the number of maximum sample points
  for i in range(sample_params[1]):
    # gets rid of corrupted data points (None and Nan entries)
    if np.any(lc[i]) == 1:
      continue
    
    # fills l_curves with non-corrupted data points
    l_curves[r*sample_params[0]+count, :, 0] = np.asarray(lc[i])
    count += 1
      
    # checks if there are enough light-curves to exit the loop
    if count == sample_params[0]:
      break      
  
  if euler_sampling:
    return l_curves, mjhd, err_mag_ml
  else:
    return l_curves

### 2. Plot properties

In [0]:
def setLabels(ax, xlabel, ylabel, ticks, legend):
    """
    Function sets the axes labels, legent size, and ticks
    """
    ax.set_xlabel(xlabel, fontsize=20)
    ax.set_ylabel(ylabel, fontsize=20)
    if legend:
        ax.legend(fontsize=14)
    ax.grid()
    if ticks:
        ax.tick_params(which='both', labelsize=18)
    return

def plotLoss(ax, history, loss_Accuracy, model_name, labels):
    """
    Function plots the loss curve for training run of x epochs
    """
    ax.plot(history[loss_Accuracy[0]], '#00a15b', lw=2, label=labels[0])
    ax.plot(history[loss_Accuracy[1]], '#fc5400', lw=2, label=labels[1])
    ax.set_title('Loss curve for %s\n'%model_name)
    setLabels(ax, 'Epoch', r'$\mathcal{L}\ \equiv$ Categorical cross entropy', ticks=True, legend=True)
    return

def plotAccuracy(ax, history, loss_Accuracy, model_name, labels):
    """
    Function plots the accuracy curve for training run of x epochs
    """
    ax.plot(history[loss_Accuracy[2]],  '#00a15b', lw=2, label=labels[0])
    ax.plot(history[loss_Accuracy[3]], '#fc5400', lw=2, label=labels[1])
    ax.set_title('Accuracy curve for %s\n'%model_name)
    setLabels(ax, 'Epoch', 'Accuracy', ticks=True, legend=True)
    return

def plotConfusionMatrix(ax, confusion_probability_matrix, output_class_names, \
                        map_choice_index):
  """
  Function plots the confusion matrix
  """
  cmap_choice = [plt.cm.Blues, plt.cm.Reds, plt.cm.Greens]
  img = ax.imshow(confusion_probability_matrix, interpolation='nearest', \
                  cmap=cmap_choice[map_choice_index])
  ax.set_title('Normalised confusion matrix')    
  plt.colorbar(img, ax=ax, label='%')
  img.set_clim(0, 100)
  
  # set ticks 
  tick_marks = np.arange(len(output_class_names))
  ax.set_xticks(tick_marks)
  ax.set_yticks(tick_marks)

  # sets tick labels
  ax.set_xticklabels(output_class_names, fontsize=18)
  ax.set_yticklabels(output_class_names, fontsize=18)
  
  # fills the matrix
  fmt = '.1f'
  thresh = confusion_probability_matrix.max() / 2.0
  for i, j in itertools.product(range(confusion_probability_matrix.shape[0]), \
                                range(confusion_probability_matrix.shape[1])):
      plt.text(j, i, format(confusion_probability_matrix[i, j], fmt),
              horizontalalignment='center',
              color='white' if confusion_probability_matrix[i, j] > thresh else 'black')    
  setLabels(ax, 'True label', 'Predicted label', ticks=False, legend=False)
  return
 

### 3. Generate training and testing data set

In [0]:
def generateTestTrain(l_curves, out_catergories, out_radii, r_0):
  """
  Function to generate test and train data sets
  @out_categories, out_radii :: arr with all the categories/radii of the light curves
  @r_0 :: array with all the entries for the scale radius of the bkg object

  Returns:
  test/train data sets into which the categories and radii of light curves are classified
  """  
  categories_idx = np.arange(len(out_catergories))
  train_idx, test_idx = train_test_split(categories_idx, test_size=0.2)
  
  # train data sets
  train_l_curves = l_curves[train_idx]
  train_radii = out_radii[train_idx]
  train_cat = out_catergories[train_idx]
  # encodes categorical integer features using a one-hot scheme
  onehot_train = tf.keras.utils.to_categorical(train_cat, num_classes = len(r_0))

  # test data sets
  test_l_curves = l_curves[test_idx]
  test_radii = out_radii[test_idx]
  test_cat = out_catergories[test_idx]
  onehot_test = tf.keras.utils.to_categorical(test_cat, num_classes = len(r_0))

  data_sets = [test_radii, train_radii, test_cat, train_cat]
  return train_l_curves, test_l_curves, data_sets, onehot_train, onehot_test